In [62]:
import torch
import numpy as np
from torch import nn
from tqdm import tqdm
from pathlib import Path

In [29]:
GD = Path('/Users/mgjeon/Library/CloudStorage/GoogleDrive-A/My Drive/Notes/Work/PINN/')

In [30]:
nf2_file = GD / Path('ar_series_377_2011-02-12T00:00:00/base/20110212_000000_TAI/extrapolation_result.nf2')

In [71]:
device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
state = torch.load(nf2_file, map_location=device)
model = nn.DataParallel(state['model'])

cube_shape = state['cube_shape']

z = cube_shape[2]
coords = np.stack(np.mgrid[:cube_shape[0]:1, :cube_shape[1]:1, :z:1], -1)

spatial_norm = 160

coords = torch.tensor(coords / spatial_norm, dtype=torch.float32)

coords_shape = coords.shape

In [72]:
cube = []
batch_size = 1000
coords = coords.view((-1, 3))
it = range(int(np.ceil(coords.shape[0] / batch_size)))
for k in tqdm(it):
    coord = coords[k * batch_size: (k + 1) * batch_size]
    coord = coord.to(device)
    coord.requires_grad = True
    cube += [model(coord).detach().cpu()]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8715/8715 [00:52<00:00, 166.84it/s]


In [73]:
cube = torch.cat(cube)
cube = cube.view(*coords_shape).numpy()
b_norm = 2500
b = cube * b_norm

In [88]:
E = ((b**2).sum(-1) / (8*np.pi)).sum()
E

208957420.0

In [78]:
npy_file = GD / Path('ar_series_377_2011-02-12T00:00:00/eval_energy/20110212_000000_TAI.npy')

In [89]:
E = float(np.load(npy_file))
E

208957520.0